In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from QOGS.optimizer.tf_adam_optimizer import AdamOptimizer
from QOGS.gate_sets import TwoModeECDGateSet
from QOGS.optimizer.GateSynthesizer import GateSynthesizer
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
# device_lib.list_local_devices()


Need tf version 2.3.0 or later. Using tensorflow version: 2.5.0-rc2



In [51]:
#Optimization of ECD Circuit parameters (betas, phis, and thetas)
N = 25
# We initialize the ECDGateSet here
gate_set_params = {
    'N_cav' : N,
    'beta_scale' : 3.0, #maximum |beta| for random initialization  
}
two_mode_ECD_gate_set = TwoModeECDGateSet(**gate_set_params)

In [53]:
#The target oscillator state.
fock = 1
psi_t = qt.tensor(qt.basis(N,fock), qt.basis(N,0)) #target state

In [54]:
#the optimization options
synth_params = {
    'gateset' : two_mode_ECD_gate_set,
    'N_blocks':6,
    'N_multistart' : 5, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 100, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'beta_scale' : 1.0, #maximum |beta| for random initialization
    'initial_states' : [qt.tensor(qt.basis(2,0),qt.basis(N,0), qt.basis(N,0))], #qubit tensor oscillator, start in |g> |0>
    'target_states' : [qt.tensor(qt.basis(2,1), psi_t)], #end in |e> |target>.
    'name' : 'Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [55]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.print_info()

(6, 5, 25, 25)
(6, 5, 25, 25)
(6, 5, 25, 25)
(6, 5, 25, 25)
N_blocks: 6
optimization_type: state transfer
optimization_masks: {'betas_rho_a': None, 'betas_rho_b': None, 'betas_angle_a': None, 'betas_angle_b': None, 'phis': None, 'etas': None, 'thetas': None}
target_unitary: None
expectation_operators: None
target_expectation_values: None
N_multistart: 5
term_fid: 0.995
dfid_stop: 1e-06
learning_rate: 0.01
epoch_size: 10
epochs: 100
name: Fock 1
filename: None
comment: 
use_phase: False
timestamps: []
do_prints: True
beta_scale: 1.0
filename: Fock 1.h5

Best circuit parameters found:
betas_a:    tf.Tensor(
[ 0.67354+0.38736j  2.19213-0.02539j -0.72955-1.23603j -1.30441-0.35424j
 -0.50907-0.45095j -2.33371+1.0207j ], shape=(6,), dtype=complex64)
betas_b:    tf.Tensor(
[ 1.24117-1.45274j -0.05917+0.43734j -0.71336+0.87766j -1.21778-2.08374j
  1.35469-1.52511j  0.46791-0.58214j], shape=(6,), dtype=complex64)
phis:    tf.Tensor([ 1.37122 -0.87407  2.88826 -1.12393 -2.55464 -0.14928], shape=

In [56]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

Start time: 2022-03-08 21:20:56
 Epoch: 0 / 100 Max Fid: 0.000021 Avg Fid: 0.000005 Max dFid: 0.000000 Avg dFid: 0.000000 Elapsed time: 0:00:01.854842 Expected remaining time: 0:00:00(6, 5, 25, 25)
(6, 5, 25, 25)
(6, 5, 25, 25)
(6, 5, 25, 25)
 Epoch: 1 / 100 Max Fid: 0.000116 Avg Fid: 0.000029 Max dFid: 0.000095 Avg dFid: 0.000023 Elapsed time: 0:00:58.189708 Expected remaining time: 1:36:00.781115

In [ ]:
#can print info, including the best circuit found.
gatesynth.print_info() 

In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
phis = best_circuit['phis']
thetas = best_circuit['thetas']
print(best_circuit)